## UNK 토큰 확인

In [19]:
lines = 'fs'
lines = lines.split()
print(len(lines[0]))

2


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_dir = "/opt/ml/input/data/train/train.tsv"
dataset = pd.read_csv(train_dir, sep='\t', header=None)
df = pd.DataFrame({'sentence':dataset[1], 'entity_01':dataset[2], 'entity_02':dataset[5], 'labels':dataset[8]})
df

,sentence,entity_01,entity_02,labels
0,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,자동차,단체:제작
1,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,27석,관계_없음
2,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,UEFA,단체:별칭
3,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,공격수,인물:직업/직함
4,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,퍼쿤 씨 인트라팃,인물:부모님
...,...,...,...,...
8995,2002년 FIFA 월드컵 사우디아라비아와의 1차전에서 독일은 8-0으로 승리하였는...,사우디아라비아,2002년,관계_없음
8996,일본의 2대 메이커인 토요타와 닛산은 시장 점유율을 높이기 위한 신차 개발을 계속하...,토요타,일본,단체:본사_국가
8997,방호의의 손자 방덕룡(方德龍)은 1588년(선조 21년) 무과에 급제하고 낙안군수로...,방덕룡,선무원종공신(宣武原從功臣),인물:직업/직함
8998,LG전자는 올해 초 국내시장에 출시한 2020년형 ‘LG 그램’ 시리즈를 이달부터 ...,LG전자,북미,관계_없음


In [3]:
sen_len=[]
for i in df['sentence']:
    sen_len.append(len(i))

In [4]:
print(f"max sentence length: {np.max(sen_len)}")
print(f"min sentence length: {np.min(sen_len)}")

max sentence length: 510
min sentence length: 15


In [5]:
import pickle as pickle
import os
import pandas as pd
import torch

from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, BertConfig

# Dataset 구성.
class RE_Dataset(torch.utils.data.Dataset):
  def __init__(self, tokenized_dataset, labels):
    self.tokenized_dataset = tokenized_dataset
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.tokenized_dataset.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

# 처음 불러온 tsv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다.
# 변경한 DataFrame 형태는 baseline code description 이미지를 참고해주세요.
def preprocessing_dataset(dataset, label_type):
  label = []
  for i in dataset[8]:
    if i == 'blind':
      label.append(100)
    else:
      label.append(label_type[i])
  out_dataset = pd.DataFrame({'sentence':dataset[1],'entity_01':dataset[2],'entity_02':dataset[5],'label':label,})
  return out_dataset

# tsv 파일을 불러옵니다.
def load_data(dataset_dir):
  # load label_type, classes
  with open('/opt/ml/input/data/label_type.pkl', 'rb') as f:
    label_type = pickle.load(f)
  # load dataset
  dataset = pd.read_csv(dataset_dir, delimiter='\t', header=None)
  # preprecessing dataset
  dataset = preprocessing_dataset(dataset, label_type)
  
  return dataset

# bert input을 위한 tokenizing.
# tip! 다양한 종류의 tokenizer와 special token들을 활용하는 것으로도 새로운 시도를 해볼 수 있습니다.
# baseline code에서는 2가지 부분을 활용했습니다.
def tokenized_dataset(dataset, tokenizer):
  concat_entity = []
  for e01, e02 in zip(dataset['entity_01'], dataset['entity_02']):
    temp = ''
    temp = e01 + '[SEP]' + e02
    concat_entity.append(temp)
  tokenized_sentences = tokenizer(
      concat_entity,
      list(dataset['sentence']),
      return_tensors="pt",
      padding=False,
      truncation=True,
      max_length=100,
      add_special_tokens=True,
      )
  return tokenized_sentences


In [6]:

MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# load dataset
train_dataset = load_data("/opt/ml/input/data/train/train.tsv")

In [7]:
tokenized = tokenizer(list(train_dataset['sentence']), add_special_tokens=True)
tokenized_length = []
for i in range(len(train_dataset['sentence'])):
    temp = ''
    temp = train_dataset['entity_01'][i] + '[SEP]' + train_dataset['entity_02'][i]
    #print(temp, train_dataset['sentence'][i])
    tokenized_sentences = tokenizer(temp, train_dataset['sentence'][i], add_special_tokens=True)

    tokenized_length.append(len(tokenized_sentences.input_ids))
print(max(tokenized_length))
print(tokenizer.decode(tokenized_sentences.input_ids))

327
[CLS] 차영수 [SEP] 의원 [SEP] 전남도의회 안전건설소방위원회 차영수 의원 ( 강진1 ) 은 지난 14일 설 명절을 앞두고 사랑의 손길이 필요한 이웃을 보살피고 따뜻한 명절분위기 조성을 위해 사회복지시설을 방문해 온정을 [UNK]. [SEP]


In [8]:
dataset_dir = "/opt/ml/input/data/test/test.tsv"
test_dataset = load_data(dataset_dir)
test_label = test_dataset['label'].values

tokenized_length = []
for i in range(len(test_dataset['sentence'])):
    temp = ''
    temp = test_dataset['entity_01'][i] + '[SEP]' + test_dataset['entity_02'][i]
    #print(temp, train_dataset['sentence'][i])
    tokenized_sentences = tokenizer(temp, test_dataset['sentence'][i], add_special_tokens=True)

    tokenized_length.append(len(tokenized_sentences.input_ids))
print(max(tokenized_length))
print(tokenizer.decode(tokenized_sentences.input_ids))


260
[CLS] 나카소네 야스히로 [SEP] 자유민주당 [SEP] 도쿄도 출신으로 세이조 대학을 졸업, 1969년 12월 중의원 의원 총선거에 자유민주당 공인으로 입후보로 출마하여 당선되었고, 1985년 12월 ~ 1986년 7월에 나카소네 야스히로 내각 ( 제2차 나카소네 내각 제2차 개조내각 ) 에서 농림수산대신을 지냈다. [SEP]


In [9]:
from transformers import ElectraModel
from tokenization_kocharelectra import KoCharElectraTokenizer

# KoCharElectra-Base
model = ElectraModel.from_pretrained("monologg/kocharelectra-base-discriminator")
tokenizer = KoCharElectraTokenizer.from_pretrained("monologg/kocharelectra-base-discriminator")


df = load_data("/opt/ml/input/data/train/train.tsv")

# MODEL_NAME = "bert-base-multilingual-cased"
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
sentence = df['sentence'].tolist()

res = []

for i in range(len(sentence)):
    res.append(tokenizer(
      sentence[i],
      return_tensors="pt",
      add_special_tokens=True,
      ))

encoded_sentences = []

for i in range(len(res)):
    s = [tokenizer.convert_ids_to_tokens(s) for s in res[i]['input_ids']]
    encoded_sentences.append(s[0])

UNK_counter = []
total_cnt = 0

for i in range(len(encoded_sentences)):
    s = encoded_sentences[i]
    sentence_cnt = 0
    flag = 0
    for j in range(len(s)):
        if 'UNK' in s[j]:
            flag=1
            sentence_cnt += 1
    if flag:
        total_cnt += 1
    UNK_counter.append(sentence_cnt)

df['UNK_cnt'] = UNK_counter

In [12]:
df.sort_values('UNK_cnt', ascending=False).head(100)

,sentence,entity_01,entity_02,label,UNK_cnt
7043,응모자는 △강재형 MBC 아나운서국 소속 국장 △공진성 HS애드(전 LG애드) 전무...,강재형,MBC 아나운서,2,17
3258,"많은 영지주의 체계들에서 (그리고 이단 연구들에서), 신은 모나드, 하나인 존재, ...",프로아르케,아이온 텔레오스,24,17
8628,'미래의 뮤지컬스타를 목표로(明日のミュージカルスターをめざして)'라는 제목의 이 다큐...,조승우,명성황후,0,17
2717,"위명대왕 이일광(威明大王 李日㫕, 베트남어: Uy Minh Đại vương Lý ...",리 태조,리 왕조,7,15
3305,스테판 라자레비치(1377년 ~ 1427년 7월 19일)는 장신의 스테판(Стева...,스테판 라자레비치,1427년,14,12
...,...,...,...,...,...
7088,"1998년 7월 1일, 정부가 공기업 민영화 방안을 발표하였고, 남해화학, 대한교과...",미래엔,교과서,6,7
6109,공화백 왕영은 1126년 조선공(朝鮮公)과 인주 이씨의 손자이자 강릉공 온(江陵公 ...,강릉공,인주 이씨,8,7
79,"유식유가행파의 수행론에 따르면, 보살이 공관(空觀)을 닦음에 있어서 초지(初地)에서...",유식유가행파,수행론,0,7
861,이어지는 종합토론 순서에서는 ▲김기영 중앙응급의료센터 강원응급의료지원센터 선임연구원...,인천광역시의료원,원장,10,7


In [13]:
df[df['UNK_cnt']>0]['sentence'].count()

798

In [ ]:
from transformers import ElectraModel
from tokenization_kocharelectra import KoCharElectraTokenizer

# KoCharElectra-Base
model = ElectraModel.from_pretrained("monologg/kocharelectra-base-discriminator")
tokenizer = KoCharElectraTokenizer.from_pretrained("monologg/kocharelectra-base-discriminator")


df = load_data("/opt/ml/input/data/train/train.tsv")

# MODEL_NAME = "bert-base-multilingual-cased"
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
sentence = df['sentence'].tolist()

res = []

for i in range(len(sentence)):
    res.append(tokenizer(
      sentence[i],
      return_tensors="pt",
      add_special_tokens=True,
      ))

encoded_sentences = []

for i in range(len(res)):
    s = [tokenizer.convert_ids_to_tokens(s) for s in res[i]['input_ids']]
    encoded_sentences.append(s[0])

UNK_counter = []
total_cnt = 0

for i in range(len(encoded_sentences)):
    s = encoded_sentences[i]
    sentence_cnt = 0
    flag = 0
    for j in range(len(s)):
        if 'UNK' in s[j]:
            flag=1
            sentence_cnt += 1
    if flag:
        total_cnt += 1
    UNK_counter.append(sentence_cnt)

df['UNK_cnt'] = UNK_counter

In [ ]:
model = ElectraModel.from_pretrained('')

In [16]:
print(sum(p.numel() for p in model.parameters() if p.requires_grad))
model = transformers.Bert.from_pretrained("bert-base-multilingual-cased")
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

94334976


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

177854978


In [ ]:
print(tokenized_train.input_ids)
tokenized_length = []
for i in tokenized_train.input_ids:
    tokenized_length.append(len(i))
print(tokenized_length)

In [10]:
print(tokenizer.decode(tokenized_train.input_ids[1]))

[CLS] 민주당 [SEP] 27석 [SEP] 선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석 ( 지역구 27석, 비례대표 30석 ) 을 획득하는 데 그쳤다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


## label 정리

In [105]:
dataset = pd.read_csv('/opt/ml/input/data/train/train.tsv', delimiter='\t', header=None)

In [106]:
with open('/opt/ml/input/data/label_type.pkl', 'rb') as f:
    label_type = pickle.load(f)

In [98]:
from collections import defaultdict
label = []
for i in dataset[8]:
    if i == 'blind':
        label.append(100)
    else:
        label.append(label_type[i])
out_dataset = pd.DataFrame({'sentence':dataset[1],'entity_01':dataset[2],'entity_02':dataset[5],'label':label,})
out_dataset.sort_values('label', ascending=False)
num_counts=[]
for i in range(42):
    num_counts.append([i, 0])
for data in out_dataset['label']:
    num_counts[data][1] += 1
num_counts.sort(key= lambda x: x[1])
low_label = []
for k, v in num_counts[:9]:
    low_label.append(k)
low_label
new_dataset =out_dataset.copy()
for i in range(9000):
    if out_dataset['label'][i] in low_label:
        new_dataset.loc[i, 'label'] = 0


6920    38
5103    38
3311    38
604     38
2440    38
        ..
2290     0
2291     0
5553     0
5552     0
1823     0
Name: label, Length: 9000, dtype: int64
5201    41
3697    41
4625    41
5631    41
6819    41
        ..
5963     0
5964     0
5965     0
5966     0
3594     0
Name: label, Length: 9000, dtype: int64


In [103]:
new_dataset.to_csv('/opt/ml/input/data/train/my_train.tsv', sep='\t', index=False)

In [104]:
dataset = pd.read_csv('/opt/ml/input/data/train/my_train.tsv', delimiter='\t')
dataset

,sentence,entity_01,entity_02,label
0,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,자동차,17
1,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,27석,0
2,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,UEFA,6
3,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,공격수,2
4,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,퍼쿤 씨 인트라팃,8
...,...,...,...,...
8995,2002년 FIFA 월드컵 사우디아라비아와의 1차전에서 독일은 8-0으로 승리하였는...,사우디아라비아,2002년,0
8996,일본의 2대 메이커인 토요타와 닛산은 시장 점유율을 높이기 위한 신차 개발을 계속하...,토요타,일본,9
8997,방호의의 손자 방덕룡(方德龍)은 1588년(선조 21년) 무과에 급제하고 낙안군수로...,방덕룡,선무원종공신(宣武原從功臣),2
8998,LG전자는 올해 초 국내시장에 출시한 2020년형 ‘LG 그램’ 시리즈를 이달부터 ...,LG전자,북미,0


In [110]:
dataset = pd.read_csv('/opt/ml/input/data/train/ner_train_ver2.tsv', delimiter='\t')
num_counts=[]
for i in range(42):
    num_counts.append([i, 0])
for data in out_dataset['label']:
    num_counts[data][1] += 1
num_counts.sort(key= lambda x: x[1])
num_counts


[[40, 1],
 [37, 3],
 [19, 4],
 [29, 5],
 [41, 5],
 [18, 7],
 [26, 8],
 [28, 9],
 [39, 9],
 [13, 11],
 [36, 11],
 [30, 12],
 [31, 15],
 [34, 15],
 [38, 18],
 [16, 23],
 [32, 26],
 [35, 27],
 [27, 36],
 [3, 44],
 [12, 45],
 [14, 45],
 [25, 52],
 [22, 56],
 [11, 58],
 [33, 67],
 [23, 79],
 [21, 84],
 [1, 92],
 [24, 103],
 [9, 115],
 [20, 136],
 [8, 164],
 [17, 169],
 [5, 186],
 [7, 209],
 [6, 231],
 [15, 335],
 [2, 553],
 [4, 679],
 [10, 815],
 [0, 4438]]

In [111]:
low_label = []
for k, v in num_counts[:9]:
    low_label.append(k)
low_label


[40, 37, 19, 29, 41, 18, 26, 28, 39]

In [113]:
new_dataset =out_dataset.copy()
for i in range(9000):
    if out_dataset['label'][i] in low_label:
        new_dataset.loc[i, 'label'] = 0
new_dataset

,sentence,entity_01,entity_02,label
0,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,자동차,17
1,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,27석,0
2,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,UEFA,6
3,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,공격수,2
4,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,퍼쿤 씨 인트라팃,8
...,...,...,...,...
8995,2002년 FIFA 월드컵 사우디아라비아와의 1차전에서 독일은 8-0으로 승리하였는...,사우디아라비아,2002년,0
8996,일본의 2대 메이커인 토요타와 닛산은 시장 점유율을 높이기 위한 신차 개발을 계속하...,토요타,일본,9
8997,방호의의 손자 방덕룡(方德龍)은 1588년(선조 21년) 무과에 급제하고 낙안군수로...,방덕룡,선무원종공신(宣武原從功臣),2
8998,LG전자는 올해 초 국내시장에 출시한 2020년형 ‘LG 그램’ 시리즈를 이달부터 ...,LG전자,북미,0


In [114]:
new_dataset.to_csv('/opt/ml/input/data/train/my_ner_train_del9.tsv', sep='\t', index=False)

In [131]:
from collections import Counter
sub0 = pd.read_csv('/opt/ml/code/prediction/submission0.csv', sep='\t')
sub1 = pd.read_csv('/opt/ml/code/prediction/submission1.csv', sep='\t')
sub2 = pd.read_csv('/opt/ml/code/prediction/submission2.csv', sep='\t')
sub3 = pd.read_csv('/opt/ml/code/prediction/submission3.csv', sep='\t')
sub4 = pd.read_csv('/opt/ml/code/prediction/submission4.csv', sep='\t')
print(sub0)
subfinal = sub0.copy()
for i in range(1000):
    temp = []
    temp.append(sub0['pred'][i])
    temp.append(sub1['pred'][i])
    temp.append(sub2['pred'][i])
    temp.append(sub3['pred'][i])
    temp.append(sub4['pred'][i])
    cnt = Counter(temp)
    print(cnt)
    subfinal['pred'][i] = max(cnt, key=cnt.get)
    print(subfinal['pred'][i])
subfinal.to_csv('./prediction/subfinal_CV.csv', index=False)

     pred
0       2
1      10
2       8
3       0
4       0
..    ...
995     0
996    10
997    10
998     0
999     0

[1000 rows x 1 columns]
Counter({0: 4, 2: 1})
0
Counter({10: 5})
10
Counter({8: 5})
8
Counter({0: 4, 17: 1})
0
Counter({0: 5})
0
Counter({33: 5})
33
Counter({7: 5})
7
Counter({0: 5})
0
Counter({4: 5})
4
Counter({20: 5})
20
Counter({0: 5})
0
Counter({0: 5})
0
Counter({0: 5})
0
Counter({8: 5})
8
Counter({0: 5})
0
Counter({0: 4, 10: 1})
0
Counter({0: 5})
0
Counter({7: 3, 0: 2})
7
Counter({0: 5})
0
Counter({10: 5})
10
Counter({15: 5})
15
Counter({5: 5})
5
Counter({0: 5})
0
Counter({27: 4, 14: 1})
27
Counter({4: 5})
4
Counter({4: 5})
4
Counter({10: 4, 0: 1})
10
Counter({0: 5})
0
Counter({0: 5})
0
Counter({0: 5})
0
Counter({0: 5})
0
Counter({0: 4, 4: 1})
0
Counter({10: 4, 12: 1})
10
Counter({8: 5})
8
Counter({0: 3, 23: 2})
0
Counter({0: 5})
0
Counter({21: 5})
21
Counter({4: 5})
4
Counter({10: 5})
10
Counter({22: 5})
22
Counter({0: 5})
0
Counter({0: 5})
0
Counter({0: 3, 10:

In [130]:
for idx in enumerate(range(5)):
    print(idx)

(0, 0)
(1, 1)
(2, 2)
(3, 3)
(4, 4)
